# Principal Component Analysis (PCA) 

~~~
Extracción de Características en Imágenes.
Master en Ciencia de Datos e Ingeniería de Computadores.
Universidad de Granada.

Fernando Pérez Bueno - fpb@ugr.es
Francisco Miguel Castro Macías - fcastro@ugr.es
Rafael Molina Soriano - rms@decsai.ugr.es
~~~

Vamos a utilizar el análisis de componentes principales (Principal Component Analysis, PCA) sobre una base de datos de caras para reducir la dimensionalidad de cada cara extrayendo un conjunto de variables latentes. Estas variables latentes podrían, con posterioridad, utilizarse, por ejemplo, en problemas de clasificación siempre que los errores de reconstrucción de las caras usando las variables latentes fuese pequeño. También podrían usarse en problemas de detección de anomalías.

El fichero ERRDfaces.mat contiene una base de datos de caras almacenada por filas en la matriz `X`. Cada fila corresponde a una cara de 32x32=1024 píxeles en niveles de gris. El número de ejemplos es 5000.

Comenzamos importando las librerías que vamos a utilizar en el desarrollo de la práctica.

In [3]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

Especificamos la ruta donde están nuestros datos y los leemos. Observa la estructura que los contiene. Lee la documentación sobre la función `loadmat`.

In [4]:
path='C://Users/cacoq/Dropbox/Trabajo/Docencia/DATCOM_ECI_22-23/'

dict_data = loadmat(path+'ERRDfaces_2021.mat')
print(dict)

<class 'dict'>


Como ves, los datos están almacenados en un diccionario. Extraemos nuestros datos usando la llave `X` y mostramos su dimensión.

In [5]:
data= dict_data['X']
data.shape

(5000, 1024)

Usaremos las 4.500 primeras caras como ejemplos de entrenamiento y las restantes 500 como test. El conjunto de test nos servirá para probar que PCA representa suficientemente bien la información como para aplicarlo a datos que no ha visto.

Dependiendo del problema podríamos usar todas las muestras como conjunto de entrenamiento. Por ejemplo, si quisiéramos detectar anomalías en el conjunto de imagenes, debemos usar un conjunto "de calibración".

> 📝 **Pregunta 1.** Completa el código para que la variable `X` siguiente contenga las 4500 primeras caras de `data` y `test` el resto.

In [ ]:
N_tr = 4500
X = 
test =

Las caras están almacenadas como un vector de rasgos, donde cada imagen es una única fila. 

> 📝 **Pregunta 2.** Redimensiona, es decir, pasa de un vector con 1024 componentes a una matriz 32x32, y muestra las 5 primeras caras del dataset. Pueden resultarte utiles las funciones `np.reshape` y `plt.imshow`. Tal vez necesites trasponer la matriz que contiene la cara.

In [ ]:
n_caras = 5
plt.figure(figsize=(15,8))

#Introduce aquí tu código

# Normalización de los datos 

Para trabajar con modelos como PCA, es necesario normalizar los datos. Recuerda una de las primeras cosas que comentó Rafa:
\begin{equation*}
\mathbf{x} \approx \mathbf{\phi} \mathbf{z} \color{red}{+ \boldsymbol{\mu}}
\end{equation*}
"Nosotros supondremos que $\color{red}{\boldsymbol{\mu}=\boldsymbol{0}}$"

> 📝 **Pregunta 3.** Usando la clase `StandardScaler` de sklearn normaliza los datos de entrenamiento, de forma que cada rasgo (de los 1024) tenga media cero. No realices el escalado de la varianza a uno. Los rasgos normalizados deberás almacenarlos en la variable `X_norm`. Ten en cuenta que más tarde tendrás que aplicar la misma normalización a los datos de test.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Introduce aquí tu código para normalizar los datos. Introduce los datos normalizados en una variable de nombre X_norm

X_norm =

> 📝 **Pregunta 4.** Escribe código, lo más eficiente posible, para comprobar que las columnas de rasgos tienen media cero.

In [ ]:
# Introduce aquí tu código

# Cálculo de las PCAs 

Una vez que has normalizado los datos. Utilizaremos `X_norm` para calcular las componentes principales. Aunque existe una función implementada en el paquete sklearn nosotros no la utilizaremos para comprender en profundidad el funcionamiento de PCAs. No se considera válido para el desarrollo de la práctica el uso de implementaciones de PCA en sklearn o cualquier otra libreria.

Supongamos que $\mathbf{X} \in \mathbb{R}^{N \times D}$ y que queremos proyectar nuestros datos en un espacio de dimensión $M$. Recuerda que el cálculo de las componentes principales viene dado por la solución al problema
$$ \min_{\mathbf{W} \in \mathbb{R}^{D \times M}, \mathbf{Z} \in \mathbb{R}^{N \times M}} \| \mathbf{X}^\top - \mathbf{W}\mathbf{Z}^\top\|_F^2.$$
En teoría hemos visto que tal solución viene dada tomando
$$ \mathbf{W} = \mathrm{Eigenvec}(\mathbf{S})_M, \quad \mathbf{Z}^\top = \mathbf{W}^\top \mathbf{X}^\top$$
donde $\mathrm{Eigenvec}(\mathbf{S})_M$ contiene los $M$ autovectores (vectores propios) con mayores autovalores (valores propios) de la matriz de covarianza muestral $\mathbf{S} = (1/N) \mathbf{X}^\top \mathbf{X}$. La reconstrucción se realiza tomando 
$$ \hat{\mathbf{X}}^\top =  \mathbf{W} \mathbf{Z}^\top.$$

Para calcular las componentes principales necesitamos calcular los autovectores de $\mathbf{X}^\top \mathbf{X}$. En teoría hemos visto que se pueden calcular a partir de la descomposición en valores singurales $\mathbf{X}^\top = \mathbf{V} \mathbf{D} \mathbf{U}^\top$. Tenemos que 
$$ \mathbf{S} = \frac{1}{N} \mathbf{V} \mathbf{D}^2 \mathbf{V}^\top,$$
lo que significa que los autovectores de $\mathbf{S}$ se encuentran en las columnas de $\mathbf{V}$ y los autovalores vienen dados por $\lambda_i = (1/N) \mathbf{D}_{ii}^2$. Recuerda que $\mathbf{D}$ es una matriz diagonal.

> 📝 **Pregunta 5.** Utiliza la descomposición por valores sigulares sobre `X_norm` o `X_norm` traspuesta usando la función `np.linalg.svd()`. Obtén los autovectores y con los valores singulares calcula los autovalores. Alternativamente, puedes calcular los autovectores y autovalores de la matriz de covarianza muestral de `X_norm` utilizando la función `np.linalg.eig()`. Tal vez sería bueno que lo hicieras con los dos métodos y comprobases que obtienes los mismos autovalores.

In [ ]:
#Introduce aqui tu codigo para calcular los autovalores y los autovectores

# Introduce en autovalores los autovalores (ordenados) de la matriz de covarianza muestral
autovalores=

# Introduce en autovectores los vectores (ordenados) que definen la transformación de PCA
autovectores =

# Número de componentes a utilizar

Vamos ahora a determinar cuantas componentes vamos a utilizar en nuestro analisis. Cada componente adicional explica parte de la varianza de nuestros datos. Queremos encontrar cuántas componentes son necesarias para representar bien nuestros datos. 

Usando los autovalores que hemos obtenido, podemos observar qué cantidad de información aporta cada una de las componentes. Las dos gráficas a continuación muestran esta información de dos formas diferentes.

In [ ]:
import matplotlib.pyplot as plt

#Hacemos uso de la suma acumulada de los autovalores para ver la varianza acumulada
cumsum=np.cumsum(autovalores)
plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.plot(cumsum/cumsum[-1])
plt.title('Varianza acumulada')
plt.subplot(2,1,2)
#El aporte de cada autovalor a la suma total, nos muestra la varianza explicada por cada componente
plt.bar(range(autovalores.shape[0]),autovalores/np.sum(autovalores))
plt.title('Varianza explicada por cada componente')
plt.show()

Aunque podemos fijar el número de componentes manualmente tras estudiar los datos, es bueno que pienses como determinarías automáticamente el número de componentes a utilizar.

> 📝 **Pregunta 6.** ¿Qué harías para determinar cuál sería un buen número de componentes? Justifica tu respuesta. No es necesario que lo implementes.

Responde aquí.

# Visualización de las autocaras

En el caso de las caras, se puede apreciar muy bien la información que captura cada una de las componentes principales visualizando las llamadas autocaras que no son más que los autovectores. Cada uno de los autovectores que hemos calculado representa elementos clave de una cara que pueden utilizarse para componer la imagen final.

> 📝 **Pregunta 7.** Igual que hiciste con las imágenes, escribe aquí código que permita visualizar los primeros 5 autovectores en forma de imagen. Es decir, los 5 autovectores asociados a los 5 autovalores mayores. Tendrás que redimensionar los vectores para darles formato de imagen.

In [ ]:
#Introduce aquí tu código para representar las 5 primeras autocaras




> 📝 **Pregunta 8.** Escribe aquí el código que permita visulizar los cinco últimos 5 autovectores en forma de imagen. Es decir, los 5 autovectores asociados a los 5 menores autovalores.

In [ ]:
#Introduce aquí tu código para representar las 5 últimas




> 📝 **Pregunta 9.** ¿Por qué son tan diferentes? 

Responde aquí.

# Cálculo de las componentes principales de los datos

Hasta ahora hemos obtenido los vectores (autovectores o autocaras) que definen el subespacio donde vamos a proyectar los datos. También hemos visto qué cantidad de varianza explica cada autovector. 

Vamos a fijar el número de autocaras a 250, es decir `n_componentes=250`.

> 📝 **Pregunta 10.** Extrae de `V` los `n_componentes` autovectores asociados a los `n_componentes` mayores autovalores y almacénalos en `W`. A continuación obtén la proyección (variables latentes) de nuestros datos y almacénalos en `Z`. Observa que lo que estamos haciendo es proyectar nuestros datos originales `X_norm` en el espacio generado por los vectores de `W`.

In [ ]:
n_componentes = 250

# W contendrá los autovectores correspondientes a las n_componentes principales

W =

# Z contendrá las proyecciones de nuestros datos en el espacio latente

Z =

print(Z.shape)

# Visualización de las primeras componentes

Podemos visualizar la primera y la segunda componentes para ver que información nos aportan sobre nuestros datos. 

> 📝 **Pregunta 11.** Crea dos gráficas separadas. Una que muestre solo la primera componente (variable latente) y otra que incluya las dos primeras (variables latentes) de todo el conjunto de entrenamiento.

In [ ]:
# Incluye aquí el código de la creación de las gráficas

> 📝 **Pregunta 12.** Comenta los resultados obtenidos.

Responde aquí. 

# Recuperacion de los datos

A partir de las variables latentes almacenadas en `Z`, cada una de sus filas contiene la representación latente de cada uno de los ejemplos (filas) en `X_norm`, podemos reconstruir los datos. Observa que hemos pasado de 1024 rasgos a `n_componentes`. 

> 📝 **Pregunta 13.** Utiliza `Z` y los autovectores en `W` para reconstruir las caras originales. Ten en cuenta que tendrás que deshacer la normalización de los datos una vez hayas recuperado la dimensionalidad original.

In [ ]:
#Introduce aquí tu código para recuperar los datos
X_norm_recovered =
X_recovered = 

> 📝 **Pregunta 14.** Muestra las 5 primeras caras de la base de datos original y su reconstrucción. Si has hecho los pasos correctamente, la reconstrucción debe ser similar al original.

In [ ]:
# Muestra 5 caras del dataset original y las mismas 5 caras reconstruidas




> 📝 **Pregunta 15.** ¿Qué ocurre si aumentamos o disminuímos el número `n_componentes`?

Reponde aquí.

# Medición del error de reconstrucción

Según el número de componentes que hayamos utilizado, habremos perdido más o menos información. Podemos comprobarlo haciendo uso del error cuadratico medio (MSE). 

> 📝 **Pregunta 16.** Usando los datos originales de entrenamiento `X` y la reconstrucción que has obtenido, calcula el MSE que has cometido con cada una de las imágenes en `X`. 

In [ ]:
#Introduce aquí tu código para calcular el MSE. Si lo has hecho bien, obtendrás un valor para cada elemento (fila) 
# de X.

mse=


> 📝 **Pregunta 17.** Haciendo uso del error cuadrático medio identifica y muestra la imagen original y la reconstruida en los siguientes casos: la imagen peor reconstruida, la imagen mejor reconstruida.

In [ ]:
# Incluye el código para encontrar en el conjunto de entregamiento las imágenes peor y mejor reconstruidas usando el
# error cuadrático medio y muéstrales.



> 📝 **Pregunta 18.** ¿Podrías explicar por qué son estas las caras que aparecen?

Responde aquí.

# Aplicando el modelo a datos nuevos 

Los autovectores o autocaras que hemos encontrado haciendo uso de `X` deberian ser suficientemente buenos para representar otros datos del mismo tipo. Vamos a comprobarlo haciendo uso del pequeño conjunto de test que separamos al principio de la práctica.

Utilizando los autovectores que ya has calculado, sigue los mismos pasos con las imagenes de test y comprueba que el funcionamiento es adecuado. 

> 📝 **Pregunta 19.** Normaliza las instancias de test.

In [ ]:
# Normalización de los datos de test. (No olvides que no puedes utilizar información obtenida del propio 
# conjunto de test)

test_norm =

> 📝 **Pregunta 20.** Obtén las variables latentes y reconstruye las caras del conjunto de test.

In [ ]:
# Obtenición de las variables latentes y reconstrucción de las caras de test.

Z_test=

Test_norm_recovered =

Test_recovered =


> 📝 **Pregunta 21.** Muestra las cinco primeras caras del conjunto de test y sus reconstrucciones.

In [ ]:
# Muestra 5 caras del conjunto de test y sus respectivas reconstrucciones





# Detección de anomalías

En el conjunto de test hay una imagen anómala que no se corresponde con el resto del dataset. ¿Como la identificarías automaticamente utilizando lo que has aprendido?

> 📝 **Pregunta 22.** Identifica y muestra la anomalía oculta en el conjunto de test. Muestra también la reconstrucción que hemos obtenido de esa anomalía.

In [ ]:
# Identifica la anomalía en el conjunto de test.



> 📝 **Pregunta 23.** ¿Qué es lo que nos permite identificar la imagen como anomalía? ¿Por qué se reconstruye de esa manera?

Responde aquí. 